# SiRIUS tsys-manual noise calculation

The following explanation is partially based on pages 188-192 in Thompson et al.'s second edition of  Interferometry and Synthesis in Radio Astronomy.

The <b>system temperature (K)</b> (with the zenith atmospheric opacity $\tau=0$)* is:
\begin{align}
T_{sys} &= T_{rx} + T_{atmos}(1 - \eta_{spill} ) + T_{cmb}\\
\end{align}
where
 - $T_{rx}$ is the receiver temperature ($K$),
 - $T_{atmos}$ is the atmospheric temperature ($K$),
 - $\eta_{spill}$ is the forward spillover efficiency,
 - $T_{cmb}$ is the cosmic microwave background temperature ($K$).
*In a future release the atmospheric opacity will be included.

The <b>effective dish area ($m^2$)</b> for baseline $ij$:
\begin{align}
A^{eff}_{ij} = \eta_a \frac{\pi d_i d_j}{4}
\end{align}
where
 - $\eta_a$ is the antenna efficiency,
 - $d_{i/j}$ is the dish diamter of the $i/j$ antenna.

The <b>rms noise level (Jy)</b> at the correlator for the $ij$ baseline for a single component of the visibility (real or complex):
\begin{align}
\sigma_{ij} = \frac{\sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff}_{i,j} \eta_q \eta_{cor} \sqrt{\Delta t \Delta \nu}}
\end{align}
where
- $k_b = 1.38064852 × 10^{-23} m^2 \: kg \: s^{-2} K^{-1}$ is Boltzmann's constant,
- $10^{26}$ is the factor that converts from $J m^{-2}$ to $Jy$.
- $\eta_q$ is the digitizer quantization efficiency,
- $\eta_{cor}$ is the correlator efficiency,
- $\Delta t$ is the correlator dump time (Seconds),
- $\Delta \nu$ is the bandwidth of a single channel (Hertz).

SiRIUS will save the following values to the main table of the measuremnet set (version 2):
\begin{align}
SIGMA_{ij} &= \sqrt{2} \sigma_{ij}, \\
WEIGHT_{ij} &= \frac{1}{2 \sigma_{ij}^2}.
\end{align}

The factor of $\sqrt{2}$ appears because of the contribution of the real and imaginary parts.

In the synthesized map, with natural weighting, the flux density of a point source for which the peak response is equal to the rms noise level is given by:
\begin{align}
S_{rms} = \sqrt{1/\left( \sum^{n_{t}} \sum^{n_{b}} \sum^{n_{chan}} \sum^{n_{pol}} \frac{1}{2 \sigma_{ij}^2} \right)} 
\end{align}
where
- $n_{t}$ is the number of time samples with dump time $\Delta t$,
- $n_{b} = \frac{n_a^2 - n_a}{2}$ is the number of baselines (no autocorrelations),
- $n_{chan}$ number of channels with bandwidth $\Delta \nu$,
- $n_{pol}$ number of instrumental polarizations used to create the image (for stokes I, $n_{pol} = 2$).

If $d=d_i = d_j$ ($\sigma_{ij}$ is no longer a function baseline) we can simplify the point source rms sensitivity to:
\begin{align}
S_{nat} = \frac{\sqrt{2} \sigma_{ij}}{\sqrt{n_{t} n_{b} n_{chan}n_{pol}}} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ (n_t \Delta t) n_b (n_{chan} \Delta \nu) n_{pol}}} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ t_{int} \nu_{int} n_b n_{pol}}} = \frac{2 \sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ t_{int} \nu_{int} n_a (n_a - 1) n_{pol}}}
\end{align}

where $t_{int} = n_{t} \Delta t$. When a weighting scheme is added:
\begin{align}
S_{wgt} = \frac{S_{nat}}{\eta_{wgt}} = S_{nat} \frac{\sqrt{\sum w_i^2}}{\sum w_i}.
\end{align}

The <b>antenna system equivalent flux density ($Jy$)</b>:
\begin{align}
SEFD &= \frac{2 k_b T_{sys}}{\eta_q A^{eff}} × 10^{26} \\
\end{align}

\begin{align}
S_{wgt} &= \frac{\sqrt{2} SEFD }{\eta_{wgt} \eta_{cor} \sqrt{ t_{int} \nu_{int} n_a (n_a - 1) n_{pol}}} .
\end{align}

\begin{align}
t_{int} =  \frac{t_{int,nat}}{\eta_{wgt}^2} = \frac{2 n_{wgt}^2 SEFD^2}{\eta_{wgt}^2 S_{wgt}^2 \eta^2_{cor} \nu_{int} n_a (n_a - 1) n_{pol}}
\end{align}
